In [9]:
import pandas as pd
import os
import numpy as np
import shutil
import glob
base_dir = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/PIPELINE/metadata"


BM_metadata_file_1 = os.path.join(base_dir, "01_Box_Evaluation_BM.xlsx")
BM_metadata_file_2 = os.path.join(base_dir, "02_Box_Evaluation_BM.xlsx")
PT_metadata_file = os.path.join(base_dir, "03_Box_Evaluation_PT.xlsx")

patients_metadata_file = os.path.join(base_dir, "20221013_Sample_cohort_with_infiltration.xlsx")
aberrations_file = os.path.join(base_dir, "202231128_Patient_Aberrations.xlsx")
clinical_data_file = os.path.join(base_dir, "20231127_Patient_Clinical_Data.xlsx")

In [10]:
#Load data 
BM_metadata_1 = pd.read_excel(BM_metadata_file_1)
BM_metadata_1 ['Tissue'] = ['BM' for i in range(len(BM_metadata_1))]
BM_metadata_2 = pd.read_excel(BM_metadata_file_2)
BM_metadata_2 ['Tissue'] = ['BM' for i in range(len(BM_metadata_2))]
PT_metadata = pd.read_excel(PT_metadata_file)
PT_metadata ['Tissue'] = ['PT' for i in range(len(PT_metadata))]

#Load technical data and eliminate unncessecary information
relevant_data = ['Slide', 'Side [R/L]', 'Date of analysis', 'Staining done by', 'Date2', 'mean IMC score', 'exclude', 'Tissue']
eval_metadata = pd.DataFrame(columns = ['Sample', 'Puncture_side', 'Staining_date', 'Stainer', 'IMC_date', 'IMC_score', 'exclude', 'Tissue'], 
    data=pd.concat([BM_metadata_1[relevant_data], BM_metadata_2[relevant_data], PT_metadata[relevant_data]], ignore_index=True).values)

#Only use the ones that worked
eval_metadata = eval_metadata.drop(eval_metadata.index[eval_metadata['exclude'] == 'yes'])
#samples_metadata = samples_metadata.dropna(subset=['exclude']) #also include samples not yet ablated
eval_metadata = eval_metadata.dropna(subset=['Sample']) #exclude empty slots
eval_metadata.reset_index(drop=True, inplace=True) #reindex after droping otherwise indices are missing

#Split into positive and negative samples - check whether clinical data and aberrations should also be determined for negative samples
#eval_metadata_neg = eval_metadata[eval_metadata['Sample'].str.contains('N', na=False)]
#eval_metadata = eval_metadata.drop(eval_metadata.index[eval_metadata['Sample'].str.contains('N', na=False)])
#eval_metadata.reset_index(drop=True, inplace=True)
eval_metadata['control'] = ['yes' if 'N' in i else 'no' for i in eval_metadata['Sample']]
eval_metadata['Sample'] = [i.split(' (')[0] if i is not np.nan else i for i in eval_metadata['Sample']]


#Load patient data and infiltration
patients_metadata = pd.read_excel(patients_metadata_file)
aberrations_metadata = pd.read_excel(aberrations_file)
clinical_metadata = pd.read_excel(clinical_data_file)

metadata = eval_metadata.reindex(columns=eval_metadata.columns.tolist() +['Study_ID', 'Infiltration_right', 'Infiltration_left', 'DE_date', 'Timepoint'])

for _i, s in enumerate(metadata['Sample']):

    if metadata['Tissue'][_i] == 'PT':
        metadata.at[_i, 'Study_ID'] = patients_metadata.loc[patients_metadata['TU (DE) ID/Biobank position'].str.contains(s, na=False), 'ID'].values[0]
        metadata.at[_i, 'Infiltration_right'] =  'NA'
        metadata.at[_i, 'Infiltration_left'] =  'NA'
        metadata.at[_i, 'DE_date'] =  patients_metadata.loc[patients_metadata['TU (DE) ID/Biobank position'].str.contains(s, na=False), 'DE_date'].values[0]
        metadata.at[_i, 'Timepoint'] = 'DE'

    else:
        tmp = [i for i in ['DE', 'RE1', 'RE2', 'REL'] if patients_metadata['BM (' + i + ') ID'].str.contains(s).any()]

        if tmp == []: 
            continue
        else:
            tmp = tmp[0]

        metadata.at[_i, 'Study_ID'] = patients_metadata.loc[patients_metadata['BM (' + tmp + ') ID'].str.contains(s, na=False), 'ID'].values[0]
        metadata.at[_i, 'Infiltration_right'] =  patients_metadata.loc[patients_metadata['BM (' + tmp + ') ID'].str.contains(s, na=False), 'BM (' + tmp + ') DTCs (Right)'].values[0]
        metadata.at[_i, 'Infiltration_left'] =  patients_metadata.loc[patients_metadata['BM (' + tmp + ') ID'].str.contains(s, na=False), 'BM (' + tmp + ') DTCs (Left)'].values[0]
        metadata.at[_i, 'DE_date'] =  patients_metadata.loc[patients_metadata['BM (' + tmp + ') ID'].str.contains(s, na=False), 'DE_date'].values[0]
        metadata.at[_i, 'Timepoint'] = tmp

#Load aberrations - Following patients are missing in aberrations: AT-0060, AT-0100, AT-0130 (double value), AT-0131
aberrations = ['1p-', '1q+', '1q-', '2p+', '3p-', '3q-', '4p-', '6q-', '7q+', 'PTPRD del', 'CDKN2A/2B del ',
       '11q-', '12q+', '14q-', '17p-', '17q+', '19p-', '19q-', '22q-', 'MYCN amp', 'TERT gain/rearr.', 'ATRX del', 'ALK amp.', 'ALK mut.']

aberrations_metadata['Patient ID'] = aberrations_metadata['Patient ID'].astype("string")
metadata['Study_ID'] = metadata['Study_ID'].astype("string")

for a in aberrations: 
    metadata[a] = [aberrations_metadata.loc[aberrations_metadata['Patient ID'].str.contains(s, na=False), a].values[0] if metadata.loc[_i, 'control']=='no' else 'NA' for _i, s in enumerate(metadata['Study_ID'])] 

metadata.rename(columns = {'1p-':'X1p_loss', '1q+':'X1q_gain', '1q-':'X1q_loss', '2p+':'X2p_gain', '3p-':'X3p_loss', '3q-':'X3q_loss', '4p-':'X4p_loss', '6q-':'X6q_loss', '7q+':'X7q_gain', 
        'PTPRD del':'PTPRD_del', 'CDKN2A/2B del ':'CDKN2AB_del', '11q-':'X11q_loss', '12q+':'X12q_gain', '14q-':'X14q_loss', '17p-':'X17p_loss', '17q+':'X17q_gain', '19p-':'X19p_loss', 
        '19q-':'X19q_loss', '22q-':'X22q_loss', 'MYCN amp':'MYCN_amp', 'TERT gain/rearr.':'TERT_rear', 'ATRX del':'ATRX_del', 'ALK amp.':'ALK_amp', 'ALK mut.':'ALK_mut'}, inplace = True)

#Load clinical data
clinical_col = ['stage4', 'rez_dat', 'prog_dat', 'event_dat', 'tod_dat1', 'efs', 'tod', 'efs_dur', 'tod_dur', 'age_at_diag', 'therapy']
for c in clinical_col: 
    metadata[c] = [clinical_metadata.loc[clinical_metadata['uid'].str.contains(s, na=False), c].values[0] if metadata.loc[_i, 'control']=='no' else 'NA' for _i, s in enumerate(metadata['Study_ID'])]

#Replace NA by empty cells"
metadata.replace('NA', np.nan, inplace=True)
metadata.replace('.', np.nan, inplace=True)

metadata.to_csv(os.path.join(base_dir, '20231128_metadata_complete.csv'))


/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
